In [1]:
import numpy as np
import scipy.io as sio
import numpy.matlib as npm
from scipy import signal
import pandas as pd
from numpy import linalg as LA
from scipy import linalg
import ipywidgets as widgets
from IPython.display import display
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema

nop=200
dim_r = 1024
Nu_elements = 32 # Variable Number of Antenna
dim_c = Nu_elements
angles = 181
buffer_size = 2*dim_r*dim_c 
y_buffer_size = 2*angles*dim_r
angle_prec=1

In [2]:
class radar1:
    def __init__(self):
        self.fc = 60*(10**9)
        self.fs = np.float32(1.76)*(10**9)
        self.L = 512
        self.PRI = 2*(10**-6)
        self.c = 3*(10**8)
        self.ts = 1/self.fs
        self.PTM_len = nop
        self.Lambda = self.c/self.fc
        self.dfd = 121
        self.im_fdmax = 12*(10**3)
        self.im_fdmin = -12*(10**3)
        
class target:
    def __init__(self, range_t, doppler_velocity, rcs_amp):
        self.range = range_t
        self.doppler_velocity = doppler_velocity
        self.rcs_amp = rcs_amp
        self.n_bodypart = 1

class antenna:
    def __init__(self, Lambda, Nelements, DOA):
        self.d_m = Lambda * np.float32(0.5)
        self.Nelements = Nelements
        self.spacing = np.arange(0, (self.Nelements) * self.d_m, self.d_m)
        self.DOA = DOA # Angle for azimuth

In [3]:
def get_received_signal_double(target, radar1, xmat1, fs, fc, antenna, n_frame):
    Mvec = radar1.PRI * np.arange(0,radar1.PTM_len)
    Marray = npm.repmat(Mvec, radar1.L, 1)
    radar1.ptx = 40
    radar1.gt = 0
    radar1.gr = 0
    rxmat = np.zeros((2*radar1.L, radar1.PTM_len), dtype=np.complex128)
    rxmat_temp = np.zeros((2*radar1.L-1, radar1.PTM_len), dtype=np.complex128)
    range_to_delay_symbols = np.floor(((2*target.range)/radar1.c)/radar1.ts)
    delay_zero = np.zeros((int(range_to_delay_symbols) - 1, int(radar1.PTM_len)))
    suffix_zero = np.zeros((int(radar1.L - range_to_delay_symbols), int(radar1.PTM_len)))
    
    # Range and doppler delay
    radar1.ptx_linear = (10**(radar1.ptx//10))*0.001
    radar1.gt_linear = (10**(radar1.gt//10))
    radar1.gr_linear = (10**(radar1.gr//10))
    radar1.radar_coeff=(np.sqrt(radar1.ptx_linear*radar1.gt_linear*radar1.gr_linear)*radar1.Lambda)/np.sqrt((4*np.pi)**3)   
    for nb in range(0,target.n_bodypart):
        velvec = target.doppler_velocity
        Doppler_ele = 2*np.pi*2*velvec/radar1.Lambda
        Doppler_Delay = np.exp(1j*(Doppler_ele*Marray))   
        temp = np.multiply(xmat1,Doppler_Delay)      
        amp = np.sqrt(target.rcs_amp)/(target.range ** 2)
        rxmat_temp = (amp*radar1.radar_coeff )*np.concatenate((delay_zero, temp, suffix_zero), dtype = np.complex128)
        rxmat[0:1023,:] = rxmat_temp

    # azimuth delay
    srx = np.zeros((2 * radar1.L, radar1.PTM_len, antenna.Nelements), dtype=np.complex128)
    lambda_m = radar1.Lambda
    antenna_matrix = np.exp(1j * 2 * np.pi * (1 / lambda_m) * np.sin(np.deg2rad(antenna.DOA)) * antenna.spacing)
    for p in range(radar1.PTM_len):
            srx[:, p, :] = np.matmul(np.transpose(rxmat[:, p][np.newaxis]),antenna_matrix[np.newaxis])
    srx_volts = np.transpose(srx, (0, 2, 1))
    return srx_volts

In [4]:
def get_received_signal_single(target, radar1, xmat1, fs, fc, antenna, n_frame):
    Mvec = radar1.PRI * np.arange(0,radar1.PTM_len)
    Marray = npm.repmat(Mvec, radar1.L, 1)
    radar1.ptx = 40
    radar1.gt = 0
    radar1.gr = 0
    rxmat = np.zeros((2*radar1.L, radar1.PTM_len), dtype=np.complex64)
    rxmat_temp = np.zeros((2*radar1.L-1, radar1.PTM_len), dtype=np.complex64)
    range_to_delay_symbols = np.floor(((2*target.range)/radar1.c)/radar1.ts)
    delay_zero = np.zeros((int(range_to_delay_symbols) - 1, int(radar1.PTM_len)))
    suffix_zero = np.zeros((int(radar1.L - range_to_delay_symbols), int(radar1.PTM_len)))
    
    # Range and doppler delay
    radar1.ptx_linear = (10**(radar1.ptx//10))*np.float32(0.001)
    radar1.gt_linear = (10**(radar1.gt//10))
    radar1.gr_linear = (10**(radar1.gr//10))
    radar1.radar_coeff=(np.sqrt(radar1.ptx_linear*radar1.gt_linear*radar1.gr_linear)*radar1.Lambda)/np.sqrt((4*np.pi)**3)   
    for nb in range(0,target.n_bodypart):
        velvec = target.doppler_velocity
        Doppler_ele = 2*np.pi*2*velvec/radar1.Lambda
        Doppler_Delay = np.exp(1j*(Doppler_ele*Marray))   
        temp = np.multiply(xmat1,Doppler_Delay)      
        amp = np.sqrt(target.rcs_amp)/(target.range ** 2)
        rxmat_temp = (amp*radar1.radar_coeff )*np.concatenate((delay_zero, temp, suffix_zero), dtype = np.complex64)
        rxmat[0:1023,:] = rxmat_temp

    # azimuth delay
    srx = np.zeros((2 * radar1.L, radar1.PTM_len, antenna.Nelements), dtype=np.complex64)
    lambda_m = radar1.Lambda
    antenna_matrix = np.exp(1j * 2 * np.pi * (1 / lambda_m) * np.sin(np.deg2rad(antenna.DOA)) * antenna.spacing)
    for p in range(radar1.PTM_len):
            srx[:, p, :] = np.matmul(np.transpose(rxmat[:, p][np.newaxis]),antenna_matrix[np.newaxis])
    srx_volts = np.transpose(srx, (0, 2, 1))
    return srx_volts

In [5]:
def get_max_y_double(y_res):
    y_antenna = np.transpose(y_res[0])
    win_hamm_col = np.hamming(181)
    awin_col = np.sum(win_hamm_col)
    power_music = 10*np.log10(np.abs(y_antenna[511:])/awin_col) + 18.706

    index_max = np.unravel_index(np.argmax(power_music, axis=None), power_music.shape)
    max_power = power_music[index_max]
    power_music[index_max] = power_music[index_max] + 5
    return max_power, power_music

In [6]:
def get_max_y_single(y_res):
    y_antenna = np.transpose(y_res[0])
    win_hamm_col = np.hamming(181)
    awin_col = np.sum(win_hamm_col)
    power_music = 10*np.log10(np.abs(y_antenna[511:])/awin_col) + np.float32(18.706)

    index_max = np.unravel_index(np.argmax(power_music, axis=None), power_music.shape)
    max_power = power_music[index_max]
    power_music[index_max] = power_music[index_max] + 5
    return max_power, power_music

In [7]:
def write_hardware_target_music(array, snr_db):
    #golden_data_t1 = open("./Results_FX/P200_Vd2_snr_"+str(snr_db)+".dat", "a")
    golden_data_t1 = open("./Results_4ms/Results_music_double/P200_Vd2_snr_"+str(snr_db)+".dat", "a")
    array_line = ' '.join(map(str, array))
    golden_data_t1.write(array_line + "\n")
    golden_data_t1.close()
    
def write_hardware_target_esprit_dpfl(array, snr_db):
    #golden_data_t1 = open("./Results_FX/P200_Vd2_snr_"+str(snr_db)+".dat", "a")
    golden_data_t1 = open("./Results_4ms/Results_esprit_double/P200_Vd2_snr_"+str(snr_db)+".dat", "a")
    array_line = ' '.join(map(str, array))
    golden_data_t1.write(array_line + "\n")
    golden_data_t1.close()
    
def write_hardware_target_esprit_spfl(array, snr_db):
    #golden_data_t1 = open("./Results_FX/P200_Vd2_snr_"+str(snr_db)+".dat", "a")
    golden_data_t1 = open("./Results_4ms/Results_esprit_single/P200_Vd2_snr_"+str(snr_db)+".dat", "a")
    array_line = ' '.join(map(str, array))
    golden_data_t1.write(array_line + "\n")
    golden_data_t1.close()

 
def write_hardware_target_esprit_dpfl_pinv(array, snr_db):
    #golden_data_t1 = open("./Results_FX/P200_Vd2_snr_"+str(snr_db)+".dat", "a")
    golden_data_t1 = open("./Results_4ms/Results_esprit_double_pinv/P200_Vd2_snr_"+str(snr_db)+".dat", "a")
    array_line = ' '.join(map(str, array))
    golden_data_t1.write(array_line + "\n")
    golden_data_t1.close()
    
def write_hardware_target_esprit_spfl_pinv(array, snr_db):
    #golden_data_t1 = open("./Results_FX/P200_Vd2_snr_"+str(snr_db)+".dat", "a")
    golden_data_t1 = open("./Results_4ms/Results_esprit_single_pinv/P200_Vd2_snr_"+str(snr_db)+".dat", "a")
    array_line = ' '.join(map(str, array))
    golden_data_t1.write(array_line + "\n")
    golden_data_t1.close()
    

In [8]:
radar1_in = radar1()
doppler_axis = np.arange(radar1_in.im_fdmin, radar1_in.im_fdmax + radar1_in.dfd, radar1_in.dfd)
velocity_axis = doppler_axis * radar1_in.Lambda/2
delay_axis = np.arange(-radar1_in.L + 1, radar1_in.L)
range_axis = ((radar1_in.c * radar1_in.ts) / 2) * delay_axis
theta_axis = np.arange(-90, 91, 1)

In [9]:
def load_xmat_double():
    # Load XMAT
    tx = sio.loadmat("tx_freq_domain.mat")["xmat"]
    xmat = np.transpose(np.transpose(tx)[:nop])
    xmat1=xmat*np.squeeze(np.tile(np.exp(1j * np.pi * np.arange(4)/ 2),(1,int(xmat.shape[0] / 4)))).reshape(-1,1)
    xmat_padded1 = np.zeros((1024, nop),np.complex128)
    xmat_padded1[0:radar1_in.L,:] = xmat1
    xmat_padded = np.zeros((Nu_elements, 1024, nop), dtype=np.complex128)
    for i in range(Nu_elements):
        xmat_padded[i] = xmat_padded1
    xmat_padded = np.transpose(xmat_padded, (2, 0, 1))
    xmat_fft_conj_tran = np.zeros(shape=np.shape(xmat_padded), dtype=np.complex128)
    for i in range(radar1_in.PTM_len):
        xmat_fft_conj_tran[i] = np.conj(np.fft.fft(xmat_padded[i]))
    n_frame = 1
    
    return xmat1, xmat_fft_conj_tran, n_frame
    
    
def load_xmat_single():
    tx = sio.loadmat("tx_freq_domain.mat")["xmat"]
    xmat = np.transpose(np.transpose(tx)[:nop])
    xmat1=xmat*np.squeeze(np.tile(np.exp(1j * np.pi * np.arange(4)/ 2),(1,int(xmat.shape[0] / 4)))).reshape(-1,1)
    xmat_padded1 = np.zeros((1024, nop),np.complex64)
    xmat_padded1[0:radar1_in.L,:] = xmat1
    xmat_padded = np.zeros((Nu_elements, 1024, nop), dtype=np.complex64)
    for i in range(Nu_elements):
        xmat_padded[i] = xmat_padded1
    xmat_padded = np.transpose(xmat_padded, (2, 0, 1))
    xmat_fft_conj_tran = np.zeros(shape=np.shape(xmat_padded), dtype=np.complex64)
    for i in range(radar1_in.PTM_len):
        xmat_fft_conj_tran[i] = np.conj(np.fft.fft(xmat_padded[i]))
    n_frame = 1
    
    return xmat1, xmat_fft_conj_tran, n_frame
    

In [10]:
def add_doppler_delay(rxmat_fft, doppler_delay):
    return np.transpose([a*b for a,b in zip(rxmat_fft, doppler_delay)])

def select_theta(avg_pre):
    return np.arange(-90, 91, avg_pre)
    
    # Method for multiplication for Diagonal elements of matrix
def diag_mat_mul(rxmat_dd, xmat_conj):
    xmat_conj_trans = np.transpose(xmat_conj)
    diag_res2 = np.sum(np.multiply(rxmat_dd, xmat_conj_trans), axis = 1)
    return diag_res2
    
def dbf(rxmat_fft, azi_delay):
    rxmat_dd=np.transpose([a*b for a,b in zip(rxmat_fft, azi_delay)])
    diag_res2_sum=np.sum(rxmat_dd,axis=1)
    return diag_res2_sum

In [11]:
def matchedfiltering_angle_v4_D(rxmat_fft, xmat_fft_conj_tran,k,r_in,IDFT,avg_pre=1):
    if(r_in>=1024):
        r_in=1023
    theta = select_theta(avg_pre)
    y = np.zeros(shape=(radar1_in.PTM_len-1,), dtype=np.complex128)
    temp = np.zeros(shape=(1, 2 * radar1_in.L), dtype=np.complex128)
    for i in range(radar1_in.PTM_len-1):
        theta_k = (2 * np.pi * np.sin(np.pi * theta[k] / 180)) / radar1_in.Lambda
        azi_delay = np.exp(-1j * theta_k * antenna_param.spacing)
        out=dbf(rxmat_fft[i], azi_delay)
        out1=np.multiply(out,xmat_fft_conj_tran[i+1])
        y[i]=np.matmul(IDFT[r_in,:][np.newaxis],np.transpose(out1[np.newaxis]))/1024 
    return y

def matchedfiltering2D(total_srx_noise, xmat_fft_conj_tran, avg_pre=1):
    rxmat_fft = np.zeros(shape=np.shape(total_srx_noise), dtype=np.complex128)
    rxmat_fft = np.fft.fft(total_srx_noise)

    theta = select_theta(avg_pre)
    y = np.zeros(shape=( len(theta), 2 * radar1_in.L), dtype=np.complex128)
    for k in range(len(theta)):
        theta_k = (2 * np.pi * np.sin(np.pi * theta[k] / 180)) / radar1_in.Lambda
        azi_delay = np.exp(-1j * theta_k * antenna_param.spacing)
        out=dbf(rxmat_fft, azi_delay)
        out1=np.multiply(out,xmat_fft_conj_tran)
        y[k][:] = np.fft.ifftshift(np.fft.ifft(out1))
    return y, theta

In [12]:
def matchedfiltering_angle_v4_S(rxmat_fft, xmat_fft_conj_tran,k,r_in,IDFT,avg_pre=1):
    if(r_in>=1024):
        r_in=1023
    theta = select_theta(avg_pre)
    y = np.zeros(shape=(radar1_in.PTM_len-1,), dtype=np.complex64)
    temp = np.zeros(shape=(1, 2 * radar1_in.L), dtype=np.complex64)
    for i in range(radar1_in.PTM_len-1):
        theta_k = (2 * np.pi * np.sin(np.pi * theta[k] / 180)) / radar1_in.Lambda
        azi_delay = np.exp(-1j * theta_k * antenna_param.spacing)
        out=dbf(rxmat_fft[i], azi_delay)
        out1=np.multiply(out,xmat_fft_conj_tran[i+1])
        y[i]=np.matmul(IDFT[r_in,:][np.newaxis],np.transpose(out1[np.newaxis]))/1024 
    return y

def matchedfiltering2D_S(total_srx_noise, xmat_fft_conj_tran, avg_pre=1):
    rxmat_fft = np.zeros(shape=np.shape(total_srx_noise), dtype=np.complex64)
    rxmat_fft = np.fft.fft(total_srx_noise)

    theta = select_theta(avg_pre)
    y = np.zeros(shape=( len(theta), 2 * radar1_in.L), dtype=np.complex64)
    for k in range(len(theta)):
        theta_k = (2 * np.pi * np.sin(np.pi * theta[k] / 180)) / radar1_in.Lambda
        azi_delay = np.exp(-1j * theta_k * antenna_param.spacing)
        out=dbf(rxmat_fft, azi_delay)
        out1=np.multiply(out,xmat_fft_conj_tran)
        y[k][:] = np.fft.ifftshift(np.fft.ifft(out1))
    return y, theta

In [13]:
def get_points_2d(N, Rmax):
    D = 2
    x = np.random.randn(N, D)
    Rmin = 1
    Rad = Rmax
    
    y = np.zeros((N, D))
    r = np.zeros(N)
    
    for n in range(N):
        dr = np.linalg.norm(x[n, :])
        U = np.random.random() ** (1 / D)
        y[n, :] = U * Rad * x[n, :] / dr
        r[n] = np.linalg.norm(y[n, :])
        
        if r[n] < Rmin:
            y[n, :] = np.full(D, np.finfo(float).tiny)
    
    x_coordinate = y[:, 0]
    y_coordinate = y[:, 1]
    
    return x_coordinate, y_coordinate

In [ ]:
# Single Target SNR Performance
Nu_elements = 32 # Variable Number of Antenna
np.random.seed(1)
radar_pos = np.array([0, 0])
snr_db = 10
N = 200
Rmax = 40
x_coordinate, y_coordinate = get_points_2d(N, Rmax)

range1 = np.round(np.sqrt((radar_pos[0] - x_coordinate) ** 2 + (radar_pos[1] - y_coordinate) ** 2))
theta1 = np.round(np.rad2deg(np.arctan2(y_coordinate, x_coordinate)))
range1[range1 == 0] = 1
x_coordinate, y_coordinate = get_points_2d(N, Rmax)
theta1[(np.where(np.logical_and(theta1>=90, theta1<=180)))]=theta1[(np.where(np.logical_and(theta1>=90, theta1<=180)))]-180
theta1[(np.where(np.logical_and(theta1>=-180, theta1<=-90)))]=theta1[(np.where(np.logical_and(theta1>=-180, theta1<=-90)))]+180
vel1 = np.random.randint(-29, 9, size=N)
rcs1 = np.random.exponential(10, size=N)
vel2 = vel1 + 4 #### velocity difference for super-resolution
rcs2 = np.random.exponential(10, size=N)
theta1[theta1 > 85] = 85
theta1[theta1 < -85] = -85
rcs1[rcs1 == 0] = 1
rcs2[rcs2 == 0] = 1
r = np.transpose(np.vstack((range1)))
r1=range1
o1=theta1
v1=vel1
v2=vel2
sigma1=rcs1
sigma2=rcs2

In [15]:
def findPeaks(arr):
    maxima_indices = argrelextrema(arr, np.greater)
    local_maxima = list(zip(maxima_indices[0], arr[maxima_indices]))
    threshold = 0.50 * np.min(arr)
    local_maxima = [tup for tup in local_maxima if tup[1] >= threshold]
    if len(local_maxima)==1:
        local_maxima.append(local_maxima[0])
    sorted_maxima = sorted(local_maxima, key=lambda x: x[1], reverse=True)
    top_two_indices = [index for index, value in sorted_maxima[:2]]
    return np.array(top_two_indices)

In [16]:
def autocorrelation_func_ED(rec_sig):
    conj_sig = np.conj(rec_sig)
    autocorrelation_matrix = []
    for i in range(len(conj_sig)):
        dat1 = conj_sig[i] * rec_sig
        autocorrelation_matrix.append(dat1)
    return np.array(autocorrelation_matrix,dtype = np.complex128)

def Phi_cal_ED(A, B,det_A,k):
    
    a11, a12 = A[0, 0], A[0, 1]
    a21, a22 = A[1, 0], A[1, 1]
    
    b11, b12 = B[0, 0], B[0, 1]
    b21, b22 = B[1, 0], B[1, 1]

    phi = np.zeros((k, k), dtype=np.complex128)
    
    # Calculate phi matrix elements
    phi[0,0] = (a22 * b11 - a12 * b21)
    phi[0,1] = (a22 * b12 - a12 * b22)
    phi[1,0] = (-a21 * b11 + a11 * b21)
    phi[1,1] = (a11 * b22 - a21 * b12)

    phi /=det_A
    return phi

def eign_cal_ED(phi_mat):
    
    trace = phi_mat[0,0] + phi_mat[1,1]
    determinant = phi_mat[0, 0] * phi_mat[1, 1] - phi_mat[0, 1] * phi_mat[1, 0]
    discriminant = trace**2 - 4 * determinant
    lambda1 = (trace + np.sqrt(discriminant)) / 2
    lambda2 = (trace - np.sqrt(discriminant)) / 2
    return np.array([lambda1,lambda2])

In [17]:
def check_and_replace(arr):
    
    if -30 <= arr[0] <= 30 and -30 <= arr[1] <= 30:
        return arr
    elif -30 <= arr[0] <= 30:
        arr[1] = arr[0]
    elif -30 <= arr[1] <= 30:
        arr[0] = arr[1]
    else:
        arr = [0, 0]
    return arr

In [18]:
def autocorrelation_func_ES(rec_sig):
    conj_sig = np.conj(rec_sig)
    autocorrelation_matrix = []
    for i in range(len(conj_sig)):
        dat1 = conj_sig[i] * rec_sig
        autocorrelation_matrix.append(dat1.astype(np.complex64))
    return np.array(autocorrelation_matrix,dtype = np.complex64)

def Phi_cal_ES(A, B,det_A,k):
    
    a11, a12 = A[0, 0], A[0, 1]
    a21, a22 = A[1, 0], A[1, 1]
    
    b11, b12 = B[0, 0], B[0, 1]
    b21, b22 = B[1, 0], B[1, 1]

    phi = np.zeros((k, k), dtype=np.complex64)
    
    # Calculate phi matrix elements
    phi[0,0] = (a22 * b11 - a12 * b21)
    phi[0,1] = (a22 * b12 - a12 * b22)
    phi[1,0] = (-a21 * b11 + a11 * b21)
    phi[1,1] = (a11 * b22 - a21 * b12)

    phi /=det_A
    return phi

def eign_cal_ES(phi_mat):
    
    trace = phi_mat[0,0] + phi_mat[1,1]
    determinant = phi_mat[0, 0] * phi_mat[1, 1] - phi_mat[0, 1] * phi_mat[1, 0]
    discriminant = trace**2 - np.complex64(4.0) * determinant
    lambda1 = (trace + np.sqrt(discriminant, dtype=np.complex64)) / np.complex64(2.0)
    lambda2 = (trace - np.sqrt(discriminant, dtype=np.complex64)) / np.complex64(2.0)
    return np.array([lambda1, lambda2], dtype=np.complex64)

In [19]:
def givensrotation(a, b):
    z = a*a + b*b
    r = np.sqrt(z.real**2 + z.imag**2)
    theta = np.arctan2(z.imag, z.real)
    hypo = np.sqrt(r) * (np.cos(theta / 2) + 1j * np.sin(theta / 2))
    if hypo == 0:
        return 1, 0
    else:
        cos1 = a/ hypo
        sin1 = -b / hypo
        return cos1, sin1

In [20]:
def subspace_gen(doppler_axis,N):
    subspace = np.zeros((1, N), dtype=np.complex128)
    for jj in range(N):
        subspace[0, jj] = np.exp(-1j*doppler_axis*jj)
    return subspace

In [21]:
def qr_givens_D(A):
    m, n = A.shape
    R = A.copy()
    Q = np.identity(m,dtype=np.complex128)
    for i in range(0, n - 1):
        for j in range(i + 1, m):
            cos, sin = givensrotation(R[i, i], R[j, i])
            R[i], R[j] = (R[i]*cos) + (R[j]*(-sin)), (R[i]*sin) + (R[j] * cos)
            Q[i], Q[j] = (Q[i]*cos) + (Q[j]*(-sin)), (Q[i]*sin) + (Q[j] * cos)
    return np.transpose(np.conj(Q)), R

def qr_givens_S(A):
    m, n = A.shape
    R = A.copy()
    Q = np.identity(m, dtype=np.complex64)
    for i in range(0, n - 1):
        for j in range(i + 1, m):
            cos, sin = givensrotation(R[i, i], R[j, i])
            R[i], R[j] = (R[i]*cos) + (R[j]*(-sin)), (R[i]*sin) + (R[j] * cos)
            Q[i], Q[j] = (Q[i]*cos) + (Q[j]*(-sin)), (Q[i]*sin) + (Q[j] * cos)
    return np.transpose(np.conj(Q)), R

In [22]:
def music_PS(rec_signal):
    rec_signal = rec_signal.astype(np.complex128)
    N = len(rec_signal)
    l = N//2
    subarr=N+1-l
    autocorrelation_matrix = np.zeros((l, l), dtype=np.complex128)
    for i in range(subarr):
        sub1 = rec_signal[i:(l+i)]
        sub2 = autocorrelation_func_ED(np.array(sub1))
        autocorrelation_matrix = autocorrelation_matrix+sub2
    autocorrelation_matrix = autocorrelation_matrix/subarr
    eign_vec, eign_val = qr_givens_D(autocorrelation_matrix)
    noise_subspace_est = np.delete(eign_vec, [0,1], axis=1)
    doppler_axis1=[-0.15079644737231,	-0.149275916527973,	-0.147755385683635,	-0.146234854839298,	-0.14471432399496,	-0.143193793150623,	-0.141673262306285,	-0.140152731461948,	-0.13863220061761,	-0.137111669773273,	-0.135591138928935,	-0.134070608084598,	-0.132550077240261,	-0.131029546395923,	-0.129509015551586,	-0.127988484707248,	-0.126467953862911,	-0.124947423018573,	-0.123426892174236,	-0.121906361329898,	-0.120385830485561,	-0.118865299641223,	-0.117344768796886,	-0.115824237952548,	-0.114303707108211,	-0.112783176263874,	-0.111262645419536,	-0.109742114575199,	-0.108221583730861,	-0.106701052886524,	-0.105180522042186,	-0.103659991197849,	-0.102139460353511,	-0.100618929509174,	-0.0990983986648364,	-0.097577867820499,	-0.0960573369761615,	-0.0945368061318241,	-0.0930162752874866,	-0.0914957444431491,	-0.0899752135988117,	-0.0884546827544742,	-0.0869341519101368,	-0.0854136210657993,	-0.0838930902214618,	-0.0823725593771244,	-0.0808520285327869,	-0.0793314976884495,	-0.077810966844112,	-0.0762904359997745,	-0.0747699051554371,	-0.0732493743110996,	-0.0717288434667622,	-0.0702083126224247,	-0.0686877817780872,	-0.0671672509337498,	-0.0656467200894123,	-0.0641261892450749,	-0.0626056584007374,	-0.0610851275563999,	-0.0595645967120625,	-0.058044065867725,	-0.0565235350233876,	-0.0550030041790501,	-0.0534824733347126,	-0.0519619424903752,	-0.0504414116460377,	-0.0489208808017003,	-0.0474003499573628,	-0.0458798191130253,	-0.0443592882686879,	-0.0428387574243504,	-0.041318226580013,	-0.0397976957356755,	-0.038277164891338,	-0.0367566340470006,	-0.0352361032026631,	-0.0337155723583257,	-0.0321950415139882,	-0.0306745106696507,	-0.0291539798253133,	-0.0276334489809758,	-0.0261129181366384,	-0.0245923872923009,	-0.0230718564479634,	-0.021551325603626,	-0.0200307947592885,	-0.0185102639149511,	-0.0169897330706136,	-0.0154692022262761,	-0.0139486713819387,	-0.0124281405376012,	-0.0109076096932638,	-0.0093870788489263,	-0.00786654800458884,	-0.00634601716025138,	-0.00482548631591392,	-0.00330495547157646,	-0.001784424627239,	-0.000263893782901543,	0.00125663706143592,	0.00277716790577338,	0.00429769875011084,	0.0058182295944483,	0.00733876043878576,	0.00885929128312322,	0.0103798221274607,	0.0119003529717981,	0.0134208838161356,	0.0149414146604731,	0.0164619455048105,	0.017982476349148,	0.0195030071934854,	0.0210235380378229,	0.0225440688821604,	0.0240645997264978,	0.0255851305708353,	0.0271056614151727,	0.0286261922595102,	0.0301467231038477,	0.0316672539481851,	0.0331877847925226,	0.03470831563686,	0.0362288464811975,	0.037749377325535,	0.0392699081698724,	0.0407904390142099,	0.0423109698585473,	0.0438315007028848,	0.0453520315472223,	0.0468725623915597,	0.0483930932358972,	0.0499136240802346,	0.0514341549245721,	0.0529546857689096,	0.054475216613247,	0.0559957474575845,	0.0575162783019219,	0.0590368091462594,	0.0605573399905969,	0.0620778708349343,	0.0635984016792718,	0.0651189325236092,	0.0666394633679467,	0.0681599942122841,	0.0696805250566216,	0.0712010559009591,	0.0727215867452965,	0.074242117589634,	0.0757626484339714,	0.0772831792783089,	0.0788037101226464,	0.0803242409669838,	0.0818447718113213,	0.0833653026556587,	0.0848858334999962,	0.0864063643443337,	0.0879268951886711,	0.0894474260330086,	0.0909679568773461,	0.0924884877216835,	0.094009018566021,	0.0955295494103584,	0.0970500802546959,	0.0985706110990334,	0.100091141943371,	0.101611672787708,	0.103132203632046,	0.104652734476383,	0.106173265320721,	0.107693796165058,	0.109214327009396,	0.110734857853733,	0.11225538869807,	0.113775919542408,	0.115296450386745,	0.116816981231083,	0.11833751207542,	0.119858042919758,	0.121378573764095,	0.122899104608433,	0.12441963545277,	0.125940166297108,	0.127460697141445,	0.128981227985783,	0.13050175883012,	0.132022289674457,	0.133542820518795,	0.135063351363132,	0.13658388220747,	0.138104413051807,	0.139624943896145,	0.141145474740482,	0.14266600558482,	0.144186536429157,	0.145707067273495,	0.147227598117832,	0.14874812896217,	0.150268659806507,	0.151789190650844]
    p_music = []
    for i in range(len(doppler_axis)):
        subspace = subspace_gen(doppler_axis1[i],l)
        ss1_nn1 = np.matmul(subspace, noise_subspace_est)
        peak1 = np.sum(ss1_nn1.real**2) + np.sum(ss1_nn1.imag**2)
        p_music.append(peak1)
    #print("PS output")
    t=np.min(p_music)/p_music
    #for i in range(len(t)):
    #    print(t[i],",")
    p_music = np.flip(np.array(p_music))
    p_music = 10*np.log10(np.min(p_music)/p_music)
    A1 = findPeaks(p_music)
    if len(A1) == 0:
        velx1 = [30.0, 30.0]
    else:
        velocity_axis = np.linspace(-30,30,200)
        velx1 = velocity_axis[A1].tolist()
        velx1.sort(reverse=True)
    return velx1[0], velx1[1], t

In [23]:
def ESPRIT_PS_S_PINV(rec_signal):
    rec_signal = rec_signal.astype(np.complex64)
    N = len(rec_signal)
    l = N//2
    k = 2
    subarr = N + 1 - l
    autocorrelation_matrix = np.zeros((l, l), dtype=np.complex64)
    for i in range(subarr):
        sub1 = rec_signal[i:(l + i)]
        sub2 = autocorrelation_func_ES(np.array(sub1))
        autocorrelation_matrix += sub2
    autocorrelation_matrix /= subarr
    eign_vec, eign_val = qr_givens_S(autocorrelation_matrix)
    S = eign_vec[:, :k]

    phi_mat = np.zeros((k, k), dtype=np.complex64)
    subA1, subB1 = (
    S[0 : autocorrelation_matrix.shape[0] - 1],
    S[1 : autocorrelation_matrix.shape[0]],
    )
    # Generate Phi matrix
    phi_mat = np.linalg.pinv(subA1) @ subB1
    phi_mat = phi_mat.astype(np.complex64)
    phi = eign_cal_ES(phi_mat)
    omega_estimates = np.angle(phi.astype(np.complex64)) / np.float32(0.005)
    omega_estimates = np.array(check_and_replace(omega_estimates), dtype=np.float32)
    omega_estimates = np.sort(omega_estimates)[::-1]
    return omega_estimates

In [24]:
def ESPRIT_PS_D_PINV(rec_signal):
    rec_signal = rec_signal.astype(np.complex128)
    N = len(rec_signal)
    l = N//2
    k = 2
    subarr = N + 1 - l
    autocorrelation_matrix = np.zeros((l, l), dtype=np.complex128)
    for i in range(subarr):
        sub1 = rec_signal[i:(l + i)]
        sub2 = autocorrelation_func_ED(np.array(sub1))
        autocorrelation_matrix += sub2
    autocorrelation_matrix /= subarr
    eign_vec, eign_val = qr_givens_D(autocorrelation_matrix)
    S = eign_vec[:, :k]

    phi_mat = np.zeros((k, k), dtype=np.complex128)
    subA1, subB1 = (
    S[0 : autocorrelation_matrix.shape[0] - 1],
    S[1 : autocorrelation_matrix.shape[0]],
    )
    # Generate Phi matrix
    phi_mat = np.linalg.pinv(subA1) @ subB1
    phi = eign_cal_ED(phi_mat)
    omega_estimates = np.angle(phi) / 0.005
    omega_estimates = check_and_replace(omega_estimates)
    omega_estimates = np.sort(omega_estimates)[::-1]
    return omega_estimates

In [25]:
def ESPRIT_PS_D(rec_signal):
    rec_signal = rec_signal.astype(np.complex128)
    N = len(rec_signal)
    l = N//2
    k = 2
    subarr = N + 1 - l
    autocorrelation_matrix = np.zeros((l, l), dtype=np.complex128)
    for i in range(subarr):
        sub1 = rec_signal[i:(l + i)]
        sub2 = autocorrelation_func_ED(np.array(sub1))
        autocorrelation_matrix += sub2
    autocorrelation_matrix /= subarr
    eign_vec, eign_val = qr_givens_D(autocorrelation_matrix)
    S = eign_vec[:, :k]

    phi_mat = np.zeros((k, k), dtype=np.complex128)
    iter1 = 0
    for i in range(0,l-1,1):
        subA1 = S[i:i+k, :]
        subB1 = S[i+1:i+k+1,:]
        if subA1.shape[0] == 2 and subB1.shape[0] == 2:
            det_A = subA1[0, 0] * subA1[1, 1] - subA1[0, 1] * subA1[1, 0]
            if det_A == 0:
                phi_mat = phi_mat
            else:
                phi_mat += Phi_cal_ED(subA1, subB1,det_A,k)
                iter1 +=1

    phi_mat /= iter1
    phi = eign_cal_ED(phi_mat)
    omega_estimates = np.angle(phi) / 0.005
    omega_estimates = check_and_replace(omega_estimates)
    omega_estimates = np.sort(omega_estimates)[::-1]
    return omega_estimates

In [26]:
def ESPRIT_PS_S(rec_signal):
    rec_signal = rec_signal.astype(np.complex64)
    N = len(rec_signal)
    l = N//2
    k = 2
    subarr = N + 1 - l
    autocorrelation_matrix = np.zeros((l, l), dtype=np.complex64)
    for i in range(subarr):
        sub1 = rec_signal[i:(l + i)]
        sub2 = autocorrelation_func_ES(np.array(sub1))
        autocorrelation_matrix += sub2
    autocorrelation_matrix /= subarr
    eign_vec, eign_val = qr_givens_S(autocorrelation_matrix)
    S = eign_vec[:, :k]

    phi_mat = np.zeros((k, k), dtype=np.complex64)
    iter1 = 0
    for i in range(0,l-1,1):
        subA1 = S[i:i+k, :]
        subB1 = S[i+1:i+k+1,:]
        if subA1.shape[0] == 2 and subB1.shape[0] == 2:
            det_A = subA1[0, 0] * subA1[1, 1] - subA1[0, 1] * subA1[1, 0]
            if det_A == 0:
                phi_mat = phi_mat
            else:
                phi_mat += Phi_cal_ES(subA1, subB1,det_A,k)
                iter1 +=1

    phi_mat /= iter1
    phi = eign_cal_ES(phi_mat)
    omega_estimates = np.angle(phi.astype(np.complex64)).astype(np.float32) / np.float32(0.005)
    omega_estimates = check_and_replace(omega_estimates)
    omega_estimates = np.sort(omega_estimates)[::-1]
    return omega_estimates

In [27]:
def ESPRIT_F_D_PINV(rec_signal1):
    vx1, vx2 = ESPRIT_PS_D_PINV(rec_signal1)
    vx3 = 0
    vx4 = 0
    return vx1, vx2, vx3, vx4

def ESPRIT_F_D(rec_signal1):
    vx1, vx2 = ESPRIT_PS_D(rec_signal1)
    vx3 = 0
    vx4 = 0
    return vx1, vx2, vx3, vx4

def ESPRIT_F_S(rec_signal1):
    vx1, vx2 = ESPRIT_PS_S(rec_signal1)
    vx3 = 0
    vx4 = 0
    return vx1, vx2, vx3, vx4

def ESPRIT_F_S_PINV(rec_signal1):
    vx1, vx2 = ESPRIT_PS_S_PINV(rec_signal1)
    vx3 = 0
    vx4 = 0
    return vx1, vx2, vx3, vx4


def MUSIC_F(rec_signal1):
    #print("INPUT")
    #print(np.real(rec_signal1))
    #print(np.imag(rec_signal1))
    vx1, vx2, p_music_PS = music_PS(rec_signal1)
    #vx3, vx4, p_music_PL = music_PS_QR(rec_signal1)
    return vx1, vx2, p_music_PS

In [28]:
import time
p_music_PS=np.zeros((12,200,200))
music_input=np.zeros((12,200,nop),'complex128')

In [29]:
ps_music = 0
ps_esprit_s = 0
ps_esprit_d = 0
ps_esprit_pinv = 0
idx = 0
for j in range(40):
    print("Round: " + str(j))
    N=5
    snr_db = 10
    signal_power = np.zeros(N)
    srx_volts_D = np.zeros((N, radar1_in.PTM_len, Nu_elements, 2 * radar1_in.L), dtype=np.complex128)
    srx_noise_D = np.zeros((N, radar1_in.PTM_len, Nu_elements, 2 * radar1_in.L), dtype=np.complex128)
    rxmat_fft_D = np.zeros(shape=np.shape(srx_noise_D[1:,:,:]), dtype=np.complex128)
    srx_volts_S = np.zeros((N, radar1_in.PTM_len, Nu_elements, 2 * radar1_in.L), dtype=np.complex64)
    srx_noise_S = np.zeros((N, radar1_in.PTM_len, Nu_elements, 2 * radar1_in.L), dtype=np.complex64)
    rxmat_fft_S = np.zeros(shape=np.shape(srx_noise_S[1:,:,:]), dtype=np.complex64)
    n1 = 1024
    w1 = np.exp(-1j * 2 * np.pi / n1)
    I, J = np.meshgrid(np.arange(1, n1 + 1), np.arange(1, n1 + 1))
    DFT1 = w1**((I - 1) * (J - 1))
    IDFT = np.transpose(np.conj(DFT1))
    mean_signal_power_D=(10*32*1*(0.005**2)*1)/(((4*np.pi)**3)*(40**4))
    mean_signal_power_S=(10*32*1*(np.float32(0.005)**2)*2)/(((4*np.pi)**3)*(40**4))
    s = 0
    
    for snr_db in [-15,-10,-5,0,5,10,15,20,25,30,35,40]:
        l = 0
        print("SNR:", snr_db)
        snr_linear = 10 ** (snr_db/10)
        noise_power_S = mean_signal_power_S/snr_linear
        noise_power_D = mean_signal_power_D/snr_linear
        for i in range(j*N,(j+1)*N):
            range_t=r1[i];
            velocity1=v1[i];
            velocity2=v2[i];
            rcs_amp1=sigma1[i];
            rcs_amp2=sigma2[i];
            DOA_angle=o1[i];
            t1 = target(range_t, velocity1, rcs_amp1)
            t2 = target(range_t, velocity2, rcs_amp2) 
            antenna_param = antenna(radar1_in.Lambda, Nu_elements, DOA_angle)
            xmat1_D, xmat_fft_conj_tran_D, n_frame_D = load_xmat_double()
            rxmat_t1_D = get_received_signal_double(t1, 
                                            radar1_in, 
                                            xmat1_D, 
                                            radar1_in.fs, 
                                            radar1_in.fc, 
                                            antenna_param, 
                                            n_frame_D)

            rxmat_t2_D = get_received_signal_double(t2, 
                                            radar1_in, 
                                            xmat1_D, 
                                            radar1_in.fs, 
                                            radar1_in.fc, 
                                            antenna_param, 
                                            n_frame_D)
            
            xmat1_S, xmat_fft_conj_tran_S, n_frame_S = load_xmat_single()
            rxmat_t1_S = get_received_signal_single(t1, 
                                            radar1_in, 
                                            xmat1_S, 
                                            radar1_in.fs, 
                                            radar1_in.fc, 
                                            antenna_param, 
                                            n_frame_S)

            rxmat_t2_S = get_received_signal_single(t2, 
                                            radar1_in, 
                                            xmat1_S, 
                                            radar1_in.fs, 
                                            radar1_in.fc, 
                                            antenna_param, 
                                            n_frame_S)
            
            rxmat_t12_D = rxmat_t1_D + rxmat_t2_D
            srx_volts_D = np.transpose(rxmat_t12_D, (2, 1, 0))
            noise_D = np.sqrt(noise_power_D) * (np.random.randn(srx_volts_D.shape[0],srx_volts_D.shape[1],srx_volts_D.shape[2]) + (1j * np.random.randn(srx_volts_D.shape[0],srx_volts_D.shape[1],srx_volts_D.shape[2])))
            srx_noise_D = srx_volts_D + noise_D
            rxmat_fft_D = np.fft.fft(srx_noise_D[1:,:,:])
            
            y_pack1_D = np.zeros(shape=(radar1_in.PTM_len,), dtype=np.complex128)
            y_p1_D = np.zeros(shape=(radar1_in.PTM_len-1,), dtype=np.complex128)
            
            rxmat_t12_S = rxmat_t1_S + rxmat_t2_S
            srx_volts_S = np.transpose(rxmat_t12_S, (2, 1, 0))
            noise_S = np.sqrt(noise_power_S) * (np.random.randn(srx_volts_S.shape[0],srx_volts_S.shape[1],srx_volts_S.shape[2]) + (1j * np.random.randn(srx_volts_S.shape[0],srx_volts_S.shape[1],srx_volts_S.shape[2])))
            srx_noise_S = srx_volts_S + noise_S
            rxmat_fft_S = np.fft.fft(srx_noise_S[1:,:,:])
            
            y_pack1_S = np.zeros(shape=(radar1_in.PTM_len,), dtype=np.complex64)
            y_p1_S = np.zeros(shape=(radar1_in.PTM_len-1,), dtype=np.complex64)
            
            y_res_D, theta_D = matchedfiltering2D(srx_noise_D[0,:,:], xmat_fft_conj_tran_D[0,0,:], avg_pre=angle_prec) 
            [a1_D, b1_D] = np.unravel_index(np.argmax(np.abs(y_res_D)), y_res_D.shape)
            mf_t1=y_res_D[a1_D,b1_D]
            if(b1_D<=512):
                b1_new_D=b1_D+512
            else:
                b1_new_D=b1_D-512
            y_p1_D = matchedfiltering_angle_v4_D(rxmat_fft_D, xmat_fft_conj_tran_D[:,0,:],a1_D,b1_new_D,IDFT)
            y_pack1_D[1:radar1_in.PTM_len]=y_p1_D
            y_pack1_D[0]=mf_t1
            music_in1_D=y_pack1_D
            music_in_D = music_in1_D.flatten()
            
            y_res_S, theta_S = matchedfiltering2D(srx_noise_S[0,:,:], xmat_fft_conj_tran_S[0,0,:], avg_pre=angle_prec) 
            [a1_S, b1_S] = np.unravel_index(np.argmax(np.abs(y_res_S)), y_res_S.shape)
            mf_t1=y_res_S[a1_S,b1_S]
            if(b1_S<=512):
                b1_new_S=b1_S+512
            else:
                b1_new_S=b1_S-512
            y_p1 = matchedfiltering_angle_v4_S(rxmat_fft_S, xmat_fft_conj_tran_S[:,0,:],a1_S,b1_new_S,IDFT)
            y_pack1_S[1:radar1_in.PTM_len]=y_p1
            y_pack1_S[0]=mf_t1
            music_in1_S=y_pack1_S
            music_in_S = music_in1_S.flatten().astype(np.complex64)
            
            music_input[s,i,:]=music_in_D
            
            start1 = time.time()
            vx1_MD, vx2_MD, p_music_PS[s,i,:] = MUSIC_F(music_in_D)
            finish1 = time.time()
            ps_music += finish1 - start1
            
            start2 = time.time()
            vx1_ED, vx2_ED, vx3_ED, vx4_ED = ESPRIT_F_D(music_in_D)
            finish2 = time.time()
            ps_esprit_d += finish2 - start2
            
            start3 = time.time()
            vx1_ES, vx2_ES, vx3_ES, vx4_ES = ESPRIT_F_S(music_in_S)
            finish3 = time.time()
            ps_esprit_s += finish3 - start3
            
            
            start4 = time.time()
            vx1_EDI, vx2_EDI, vx3_EDI, vx4_EDI = ESPRIT_F_D_PINV(music_in_D)
            finish4 = time.time()
            ps_esprit_pinv += finish4 - start4
            
            start5 = time.time()
            vx1_ESI, vx2_ESI, vx3_ESI, vx4_ESI = ESPRIT_F_S_PINV(music_in_S)
            finish5 = time.time()
            ps_esprit_pinv += finish5 - start5
            
            idx += 1
            l+=1
            
            hw_MD = np.array([velocity2,velocity1,vx1_MD, vx2_MD])
            hw_ES = np.array([velocity2,velocity1,vx1_ES, vx2_ES])
            hw_ED = np.array([velocity2,velocity1,vx1_ED, vx2_ED])
            hw_EDI = np.array([velocity2,velocity1,vx1_EDI, vx2_EDI])
            hw_ESI = np.array([velocity2,velocity1,vx1_ESI, vx2_ESI])
            write_hardware_target_music(hw_MD, snr_db)
            write_hardware_target_esprit_spfl(hw_ES, snr_db)
            write_hardware_target_esprit_dpfl(hw_ED, snr_db)
            write_hardware_target_esprit_dpfl_pinv(hw_EDI, snr_db)
            write_hardware_target_esprit_spfl_pinv(hw_ESI, snr_db)
        s=s+1
        
    s = 0

Round: 0
SNR: -15


C:\Users\Samarth\AppData\Local\Temp\ipykernel_35504\1468755123.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y[i]=np.matmul(IDFT[r_in,:][np.newaxis],np.transpose(out1[np.newaxis]))/1024


SNR: -10
SNR: -5
SNR: 0
SNR: 5
SNR: 10
SNR: 15
SNR: 20
SNR: 25
SNR: 30
SNR: 35
SNR: 40
Round: 1
SNR: -15
SNR: -10
SNR: -5
SNR: 0
SNR: 5
SNR: 10
SNR: 15
SNR: 20
SNR: 25
SNR: 30
SNR: 35
SNR: 40
Round: 2
SNR: -15
SNR: -10
SNR: -5
SNR: 0
SNR: 5
SNR: 10
SNR: 15
SNR: 20
SNR: 25
SNR: 30
SNR: 35
SNR: 40
Round: 3
SNR: -15
SNR: -10
SNR: -5
SNR: 0
SNR: 5
SNR: 10
SNR: 15
SNR: 20
SNR: 25
SNR: 30
SNR: 35
SNR: 40
Round: 4
SNR: -15
SNR: -10
SNR: -5
SNR: 0
SNR: 5
SNR: 10
SNR: 15
SNR: 20
SNR: 25
SNR: 30
SNR: 35
SNR: 40
Round: 5
SNR: -15
SNR: -10
SNR: -5
SNR: 0
SNR: 5
SNR: 10
SNR: 15
SNR: 20
SNR: 25
SNR: 30
SNR: 35
SNR: 40
Round: 6
SNR: -15
SNR: -10
SNR: -5
SNR: 0
SNR: 5
SNR: 10
SNR: 15
SNR: 20
SNR: 25
SNR: 30
SNR: 35
SNR: 40
Round: 7
SNR: -15
SNR: -10
SNR: -5
SNR: 0
SNR: 5
SNR: 10
SNR: 15
SNR: 20
SNR: 25
SNR: 30
SNR: 35
SNR: 40
Round: 8
SNR: -15
SNR: -10
SNR: -5
SNR: 0
SNR: 5
SNR: 10
SNR: 15
SNR: 20
SNR: 25
SNR: 30
SNR: 35
SNR: 40
Round: 9
SNR: -15
SNR: -10
SNR: -5
SNR: 0
SNR: 5
SNR: 10
SNR: 15
SNR: 20


KeyboardInterrupt: 